# Lab 3 - Ethers


## 1. Setup

### Step 1: Copy content of lab-2 into lab-3


In [ ]:
cp -rp ~/github/suss/fin579/lab-2/. ~/github/suss/fin579/lab-3
cd ~/github/suss/fin579/lab-3

## 2. Lab 3a: Transferring ETH on Development Network

### Step 1. Start Local Network

In [ ]:
hh node

### Step 2. Launch Hardhat Console

Open a terminal and launch the console.

```zsh
hh console --network localhost
```

### Step 1. Load the ethers plugin.

```js
> const { ethers } = require('hardhat');
```

### Step 2. Load the accounts.

```js
> accounts = await ethers.getSigners();
```

### Step 3. Get ETH balance of accounts[0]

```js
> before = await ethers.provider.getBalance(accounts[0].address);
//BigNumber { value: "10000000000000000000000" }
```

### Step 4. Send 0.1 ETH to accounts[1];

```js
> response = await accounts[0].sendTransaction({
    to: accounts[1].address,
    value:ethers.utils.parseUnits('0.1','ether')
});
> receipt = await response.wait();
```

### Step 5. Check the final wallet balance

```js
> after = await ethers.provider.getBalance(accounts[0].address);
```

### Step 6. Compare balance before and after
```js
> before
BigNumber { value: "10000000000000000000000" }
> after
BigNumber { value: "9999899981625000000000" }
> deduction = before.sub(after)
BigNumber { value: "100018375000000000" }
> ethers.utils.formatUnits(deduction,"ether")
'0.100018375'
```

### Question 1

You are sending `0.1` ETH but the total deduction from your wallet is `0.100018375`. How do you account for the additional cost of `0.000018375` ETH being deducted?

**ANSWER:**

```js
> response.gasPrice
//BigNumber { value: "875000000" }
> response.gasUsed
//BigNumber { value: "21000" }
> receipt.gasUsed * response.gasPrice
18375000000000
```

### Question 2

What is the cost of transaction in USD if ETHUSD is $4000?

**ANSWER:**

```js
> ethers.utils.formatUnits('18375000000000','ether')
'0.000018375'
```


## Lab 3b: Transferring ETH on Testnet

In this lab, you will signup an account with a third-party node provider that provide access to Ethereum Clients connected to public network. We then obtain some test ETH and use it to test the sending of ETH transactions on testnet. In theory, you can install your own Ethereum Client running a public node. But for practicality sake, most will connect via an external party that already have nodes running on the live networks. For this course, we will be using Alchemy as the node provider and Goerli for our testnet.

### Step 1: Signup for an account with Alchemy

You will need to create an account with Alchemy in order to use

For the deployment to Rinkeby testnet, you will require an account with a Web3 Provider, eg. Alchemy and obtain an API key.

a) Signup an account with Alchemy (https://www.alchemy.com/)

b) Apply for an API Key for the `Goerli` Testnet. The Api URL with API Key should look something like this "https://eth-goerli.g.alchemy.com/v2/123abc123abc123abc123abc123abcde"

### Step 2: Update .env with API Key

a) In Lab 2, you have created the `.env` for storing secrets. In this lab, we will update the `.env` file with another secret `FIN579_ALCHEMY_URLAPIKEY` containing the Alchemy URL and API key.

```.env
FIN579_MNEMONIC=replace this with your passphrase
FIN579_ALCHEMY_URLAPIKEY=replace this with Alchemy URL and API Key
```

b) Update hardhat.config.js with a network entry for Goerli and add the environment variable in the `url` property. This will allow us to switch easily from HH Network to Goerli via the `--network` option.

```js
require("@nomiclabs/hardhat-ethers");
require("dotenv").config();
module.exports = {
    solidity: "0.8.8",
    networks: {
        goerli: {
            chainId: 5,
            url: process.env.FIN579_ALCHEMY_URLAPIKEY,
            accounts: {
                mnemonic: process.env.FIN579_MNEMONIC,
            },
        },
        hardhat: {
            accounts: {
                mnemonic: process.env.FIN579_MNEMONIC,
            },
        },
    },
};
```

### Step 3: Connect Metamask to Goerli

Open your Metamask wallet and change the Network to `Goerli Test Network`

### Step 4: Request for test ETH

a) Go to https://goerlifaucet.com/ 

b) Login with your Alchemy account. 

c) Provide your wallet address to receive Goerli ETH.

**d) You need to have at least 0.001 ETH (~SGD2) in your mainnet wallet so that Goerli know its not a spam.**

e) Verify that you have received the ETH in Metamask. 

You can repeat the same steps **Lab 3a** to transfer and check your ETH balance by connecting the HH console to Goerli by using `hh console --network goerli`.

### Step 5: Create a transfer ETH script

a) Create `send-eth.js`.

Using the HH console allow us to test basic tasks but when dealing with more complex tasks, it may be easier to write a script instead.

```javascript
const { ethers } = require("hardhat");

(async () => {
    let accounts = await ethers.getSigners();
    let before = await ethers.provider.getBalance(accounts[0].address);
    console.log(`before: ${accounts[0].address} has ${before} ETH`);

    let qty = ethers.utils.parseEther("0.001", "ether");
    console.log(`send: ${qty} ETH to ${accounts[1].address}`);

    let response = await accounts[0].sendTransaction({
        to: accounts[1].address,
        value: qty,
    });

    console.log(`send: txHash = ${response.hash}`);
    console.log(`send: gasPrice = ${response.gasPrice}`);
    let receipt = await response.wait();
    console.log(`send: gasUsed = ${receipt.gasUsed}`);
    let after = await ethers.provider.getBalance(accounts[0].address);
    console.log(`after: ${accounts[0].address} has ${after} ETH`);
    let deduction = before.sub(after);
    console.log(`deduction: ${deduction}`);
})().catch((err) => console.error(err));
```


### Step 6. Execute the script on Goeli

In [ ]:
hh run send-eth.js --network goerli

### Step 7. Verify on Etherscan

Go to `https://goerli.etherscan.io/` and enter your transaction hash. Eg. https://goerli.etherscan.io/tx/0xc288df03888fe4258c13b330d3be43a90af24ae8d74590976e5ddd9faa2e7f23

This will confirm that the transaction you sent using the script has been mined on the public network.

### Step 8. Check your metamask balance

Check your metamask wallet to confirm the amount has been transferred from accounts[0] to accounts[1].

### Step 9. Compare the gas price

Are the gas price equal for transaction on local network vs testnet?
Which is higher? Why?


## Lab 3c: Handling Slow Transactions

In Lab3b, you may encounter a long delay because the gasPrice you are using is too low. If a transaction was sent with too low a gas price then it may not be picked up by miners. Even if you try to send a transaction with a higher gasPrice, the new transaction will not be processed until the previous transaction is mined. This is because all transactions sent from an account has a unique running number called a **transaction nonce**. If the one transaction is not mined, then any subsequent transactions after that will wait indefinitely, even if the latter has a higher gasPrice.

In this lab, we will send a low gas fee transaction that will jam up all your transactions. Then we will study how we can unjam the sequence. If your transaction in Lab-3b is still not being mined, then you do not have to proceed with this step. But if you transaction in Lab-2b is already mined, then continue with the instruction.

### Step 1. Create a script called `jam-txn.js`.


```js
const { ethers } = require("hardhat");

(async () => {
    let accounts = await ethers.getSigners();
    let before = await ethers.provider.getBalance(accounts[0].address);
    console.log(`before: ${accounts[0].address} has ${before} ETH`);

    let qty = ethers.utils.parseEther("0.001", "ether");
    console.log(`send: ${qty} ETH to ${accounts[1].address}`);

    let response = await accounts[0].sendTransaction({
        to: accounts[1].address,
        value: qty,
        gasPrice: 2000
    });

    console.log(`send: txHash = ${response.hash}`);
    console.log(`send: gasPrice = ${response.gasPrice}`);
    let receipt = await response.wait();
    console.log(`send: gasUsed = ${receipt.gasUsed}`);
    let after = await ethers.provider.getBalance(accounts[0].address);
    console.log(`after: ${accounts[0].address} has ${after} ETH`);
    let deduction = before.sub(after);
    console.log(`deduction: ${deduction}`);
})().catch((err) => console.error(err));
```
The main difference in this script is the declaration of gasPrice of 2000 wei which will likely cause the transaction to be in pending.

```js
let response = await accounts[0].sendTransaction({
    to: accounts[1].address,
    value: qty,
    gasPrice: 2000,
});
```

### Step 2. Run `jam-txn.js`

In [ ]:
hh run jam-txn.js --network goerli

### Step 3. Check transaction

Get the transaction hash from the output and check the transaction status in http://rinkeby.etherscan.io in your browser by replacing the path \<replace-this-with-the-transaction-hash>.

    ```browser
    https://goerli.etherscan.io/tx/<replace-this-with-the-transaction-hash>
    ```

    - The Status should show **Pending**.

### Step 4. Create a script called "unjam-txn.js"

```js
const { ethers } = require("hardhat");

(async () => {
    let accounts = await ethers.getSigners();
    let before = await ethers.provider.getBalance(accounts[0].address);
    console.log(`before: ${accounts[0].address} has ${before} ETH`);

    let qty = ethers.utils.parseEther("0.001", "ether");
    console.log(`send: ${qty} ETH to ${accounts[1].address}`);

    let nonce = await ethers.provider.getTransactionCount(accounts[0].address);
    let response = await accounts[0].sendTransaction({
        to: accounts[1].address,
        value: qty,
        gasPrice: 5_000_000_000,
        nonce: nonce,
    });

    console.log(`send: txHash = ${response.hash}`);
    console.log(`send: gasPrice = ${response.gasPrice}`);
    let receipt = await response.wait();
    console.log(`send: gasUsed = ${receipt.gasUsed}`);
    let after = await ethers.provider.getBalance(accounts[0].address);
    console.log(`after: ${accounts[0].address} has ${after} ETH`);
    let deduction = before.sub(after);
    console.log(`deduction: ${deduction}`);
})().catch((err) => console.error(err));

```

#### a) The script first finds the last transaction nonce.

```js
let nonce = await ethers.provider.getTransactionCount(accounts[0].address);
let response = await accounts[0].sendTransaction(...
```

This line of code is used to indirectly find the nonce of the jammed transaction using **provider.getTransactionCount()**. Since the nonce starts from 0, therefore the total number of transactions sent is equivalent to the nonce of the jammed transaction.

**NOTE:** If the jammed nonce is 45, then any transactions sent with a nonce of 44 and below will throw a `nonce has already been used` error. Any transactions with nonce above 45 will be waiting.

#### b) It then resend a transaction using the same nonce but with a higher gasPrice

```js
let response = await accounts[0].sendTransaction({
    to: accounts[1].address,
    value: 1,
    gasPrice: 5_000_000_000,
    nonce: nonce,
});
```

Technically, once a transaction is sent, it cannot be cancelled since it was broadcasted to the network. To artificially cancel a transaction, you will need to send a transaction with the `same nonce` but with a `higher gasPrice`. This is to allow the latter to overtake the former in priority of being mined.

You can set the nonce of a transaction explicitly when sending a transaction by using the **nonce** property in the **transaction request**.

The code snippet is resending the transaction with a gasPrice of 50 gwei (which is considerably high).
